In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit.circuit.library import QFT
from qiskit.compiler import transpile
from qiskit.transpiler import CouplingMap
from qiskit_aer import AerSimulator, QasmSimulator
from qiskit.visualization import plot_distribution
from fractions import Fraction
from copy import deepcopy
import numpy as np

In [2]:
service = QiskitRuntimeService(
    channel="ibm_quantum",
    token="0a2449f2c0070d110881d23f1901b55b4fa28bb4979ef64088d971415bc21f09623ab31a26458d0d53d2e563bb5d74af854c90ead93b2bda8f70f81cc0066ee8",
)

In [3]:
def carry():
    qc = QuantumCircuit(4, name="carry")
    qc.ccx(1, 2, 3)
    qc.cx(1, 2)
    qc.ccx(0, 2, 3)
    return qc.to_gate()

In [4]:
def sum():
    qc = QuantumCircuit(3, name="sum")
    qc.cx(1, 2)
    qc.cx(0, 2)
    return qc.to_gate()

In [5]:
def adder(n):

    qc = QuantumCircuit(3 * n + 1, name="adder")

    sc = 0
    for i in range(n - 1):
        qc.append(carry(), list(range(sc, sc + 4)))
        sc += 3

    qc.append(carry(), list(range(sc, sc + 4)))

    for i in range(n):
        if i == 0:
            qc.cx(sc + 1, sc + 2)
        qc.append(sum(), list(range(sc, sc + 3)))
        sc -= 3
        if sc < 0:
            break
        qc.append(carry().inverse(), list(range(sc, sc + 4)))

    return qc.to_gate()

In [6]:
# 0 <= a, b < N
# (a + b) mod N
def adder_mod(indices):

    c, a, b, N1, N2, t = indices
    n = len(a)
    qubits = 6 * n + 2
    qc = QuantumCircuit(qubits, name="adder_mod")

    # adder
    lines = []
    for i in range(n):
        lines.append(c[i])
        lines.append(a[i])
        lines.append(b[i])
    lines.append(c[-1])
    qc.append(adder(n), lines)

    # subtractor
    lines = []
    for i in range(n):
        lines.append(c[i])
        lines.append(N1[i])
        lines.append(b[i])
    lines.append(c[-1])
    qc.append(adder(n).inverse(), lines)

    # check overflow
    qc.x(c[-1])
    qc.cx(c[-1], t[0])
    qc.x(c[-1])

    # reset N1 to 0 if t is 1
    for i in range(n):
        qc.ccx(t[0], N2[i], N1[i])

    # adder
    qc.append(adder(n), lines)

    # restore N1 to original value if t is 1
    for i in range(n):
        qc.ccx(t[0], N2[i], N1[i])

    # subtractor
    lines = []
    for i in range(n):
        lines.append(c[i])
        lines.append(a[i])
        lines.append(b[i])
    lines.append(c[-1])
    qc.append(adder(n).inverse(), lines)

    # restore t to 0
    qc.cx(c[-1], t[0])

    # adder
    qc.append(adder(n), lines)

    return qc.to_gate()

In [7]:
# a*x mod N
def mult_mod(indices):

    c, a, b, N1, N2, t, r, x = indices
    n = len(a)
    qubits = 7 * n + 2
    qc = QuantumCircuit(qubits, name="mult_mod")

    # default order: (a + b) mod N
    default = [c, a, b, N1, N2, t, r]
    reg = []
    for i in default:
        for j in i:
            reg.append(j)

    # (a + r) mod N
    order = [c, a, r, N1, N2, t]
    U = adder_mod(order).control(1)
    qc.append(U, [x[0]] + reg)

    iterations = 1
    for i in range(1, n):
        for _ in range(iterations):
            # (a + b) mode N
            order = [c, a, b, N1, N2, t]
            U = adder_mod(order)
            qc.append(U, reg)
        iterations *= 2
        # (b + r) mod N
        order = [c, b, r, N1, N2, t]
        U = adder_mod(order).control(1)
        qc.append(U, [x[i]] + reg)

    return qc.to_gate()

In [8]:
def calculate(
    a: str = "",
    b: str = "",
    N: str = "",
    x: str = "",
    operation: str = "add",
    power: int = 1,
    ancilla: int = 1,
):

    if operation not in ["add", "add_mod", "mult_mod", "exp_mod", "factor"]:
        raise ValueError("Invalid operation")

    if operation == "mult_mod":
        b = deepcopy(a)

    if operation == "exp_mod":
        b = deepcopy(a)

    if operation == "factor":
        b = deepcopy(a)

    n = max(len(a), len(b), len(N))
    a = a.zfill(n)
    b = b.zfill(n)
    N = N.zfill(n)
    x = x.zfill(n)

    qubits = 7 * n + 2

    c_qubits = list(range(n + 1))  # n + 1 qubits
    a_qubits = list(range(n + 1, 2 * n + 1, 1))  # n qubits
    b_qubits = list(range(2 * n + 1, 3 * n + 1, 1))  # n qubits
    N1_qubits = list(range(3 * n + 1, 4 * n + 1, 1))  # n qubits
    N2_qubits = list(range(4 * n + 1, 5 * n + 1, 1))  # n qubits
    t = [5 * n + 1]  # 1 qubit
    r = list(range(5 * n + 2, 6 * n + 2, 1))  # n qubits
    x_qubits = list(range(6 * n + 2, 7 * n + 2, 1))  # n qubits

    if operation == "factor":
        ancilla_reg = QuantumRegister(ancilla)
        qreg = QuantumRegister(qubits)
        creg = ClassicalRegister(ancilla)
        qc = QuantumCircuit(qreg, ancilla_reg, creg)
        qc.h(ancilla_reg)
    else:
        if operation == "add":
            creg = ClassicalRegister(n + 1)
        else:
            creg = ClassicalRegister(n)
        qreg = QuantumRegister(qubits)
        qc = QuantumCircuit(qreg, creg)

    # c
    ptr = n + 1

    # a
    for i in range(n):
        if a[n - i - 1] == "1":
            qc.x(ptr)
        ptr += 1

    # b
    for i in range(n):
        if b[n - i - 1] == "1":
            qc.x(ptr)
        ptr += 1

    # N1
    for i in range(n):
        if N[n - i - 1] == "1":
            qc.x(ptr)
        ptr += 1

    # N2
    for i in range(n):
        if N[n - i - 1] == "1":
            qc.x(ptr)
        ptr += 1

    # t
    ptr += 1

    # r
    ptr += n

    # x
    for i in range(n):
        if x[n - i - 1] == "1":
            qc.x(ptr)
        ptr += 1

    if operation == "add":
        lines = []
        for i in range(n):
            lines.append(c_qubits[i])
            lines.append(a_qubits[i])
            lines.append(b_qubits[i])
        lines.append(c_qubits[-1])
        qc.append(adder(n), lines)
        qc.measure(b_qubits + [c_qubits[-1]], creg)

    if operation == "add_mod":
        indices = [c_qubits, a_qubits, b_qubits, N1_qubits, N2_qubits, t, r]
        reg = []
        for i in indices:
            for j in i:
                reg.append(j)
        order = [c_qubits, a_qubits, b_qubits, N1_qubits, N2_qubits, t]
        qc.append(adder_mod(order), reg)
        qc.measure(b_qubits, creg)

    if operation == "mult_mod":
        indices = [c_qubits, a_qubits, b_qubits, N1_qubits, N2_qubits, t, r, x_qubits]
        qc.append(mult_mod(indices), qreg)
        qc.measure(r, creg)

    # first iteration:
    # |x = 1> -> |r = a mod N>
    # second iteration:
    # |x = a mod N> -> |r = a^2 mod N>
    # same x = r of last iteration
    # a mod N is stored in r -> move to a and reset r
    # reset b and reinitialize it to original value of a
    # reset c
    if operation == "exp_mod":
        for p in range(power):
            if p != 0:
                qc.reset(c_qubits)
                qc.reset(b_qubits)
                for i in range(n):
                    qc.cx(a_qubits[i], b_qubits[i])
                qc.swap(x_qubits, r)
                qc.reset(r)
            indices = [
                c_qubits,
                a_qubits,
                b_qubits,
                N1_qubits,
                N2_qubits,
                t,
                r,
                x_qubits,
            ]
            qc.append(mult_mod(indices), qreg)
        qc.measure(r, creg)

    if operation == "factor":
        for anc in range(ancilla):
            for p in range(2**anc):
                if p != 0:
                    qc.reset(c_qubits)
                    qc.reset(b_qubits)
                    for i in range(n):
                        qc.cx(a_qubits[i], b_qubits[i])
                    qc.swap(x_qubits, r)
                    qc.reset(r)
                indices = [
                    c_qubits,
                    a_qubits,
                    b_qubits,
                    N1_qubits,
                    N2_qubits,
                    t,
                    r,
                    x_qubits,
                ]
                U = mult_mod(indices).control(1)
                qc.append(U, [ancilla_reg[anc]] + [*qreg])
        qc.append(QFT(ancilla).inverse(), ancilla_reg)
        qc.measure(ancilla_reg, creg)

    return qc

In [9]:
# params = {"a": "011", "b": "110", "operation": "add"}  # result = 1001
# params = {"a": "011", "b": "110", "N": "111", "operation": "add_mod"}  # result = "010"
# params = {"a": "101", "x": "010", "N": "111", "operation": "mult_mod"}  # result = "011"
# params = {"a": "1100", "x": "1", "N": "1110", "operation": "exp_mod", "power": 6}

In [10]:
# qc = calculate(**params)

In [11]:
# simulator = AerSimulator()
# simulator = QasmSimulator()
# qc = transpile(qc, simulator, optimization_level=3)
# job = simulator.run(qc, shots=1)
# counts = job.result().get_counts(qc)
# plot_distribution(counts)

In [12]:
# 12**6 % 14

In [13]:
def order_finding(qc, ancilla, N):

    # backend = service.least_busy()
    backend = service.get_backend("ibm_brisbane")
    print(backend.name)
    shots = 1024
    isa_circuit = transpile(qc, backend)
    sampler = Sampler(backend)
    job = sampler.run([(isa_circuit,)], shots=shots)
    bits = job.result()[0].data.meas
    counts = bits.get_counts()

    # simulator = AerSimulator()
    # isa_circuit = transpile(qc, simulator, coupling_map=CouplingMap().from_full(34))
    # result = simulator.run(isa_circuit).result()
    # counts = result.get_counts()

    highest_probability_outcome = max(counts, key=counts.get)
    phase = int(highest_probability_outcome, 2) / 2**ancilla  # theta = s / r
    r = Fraction(phase).limit_denominator(N).denominator
    return r

In [14]:
def factorization(ancilla, N):
    Zn = [i for i in range(2, N)]
    Zn_star = [i for i in Zn if np.gcd(i, N) == 1]
    i = 0
    while True:
        i += 1
        # a = np.random.choice(Zn)
        a = np.random.choice(Zn_star)
        print(f"ATTEMPT {i}: a = {a}")
        d = np.gcd(a, N)
        if d >= 2:
            return d, N // d
        qc = calculate(
            a=np.binary_repr(a),
            x="1",
            N=np.binary_repr(N),
            operation="factor",
            ancilla=ancilla,
        )
        r = order_finding(qc, ancilla, N)
        if r % 2 == 0:
            d = np.gcd(a ** (r // 2) - 1, N)
            if d >= 2:
                print(f"order of {a} mod {N} is {r}")
                return d, N // d
        print("FAILED")

In [15]:
# N = 15
# factors = factorization(ancilla=4, N=N)
# print(f"factors of {N}: {factors}")

In [16]:
N = 51
factors = factorization(ancilla=6, N=N)
print(f"factors of {N}: {factors}")

ATTEMPT 1: a = 32
